In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

spark = SparkSession.builder.getOrCreate()

# Use Unity Catalog (NO MORE /mnt/ paths!)
CATALOG = "workspace"
SCHEMA = "fca_regulatory"

# ---- SILVER: VALIDATED TRANSACTIONS ----
# Read from bronze table
df_txns = spark.table(f"{CATALOG}.{SCHEMA}.bronze_daily_transactions")

# Apply validation rules
df_valid_txns = df_txns \
    .filter(col("amount") > 0) \
    .filter(col("currency").isin("GBP", "USD", "EUR"))

# Write to silver table
df_valid_txns.write.format("delta").mode("overwrite") \
    .saveAsTable(f"{CATALOG}.{SCHEMA}.silver_validated_transactions")

print(f"✅ Created silver_validated_transactions")
print(f"📊 Valid transactions: {df_valid_txns.count()}")

# ---- CUSTOMER 360 (SCD Type 2) ----
# Read from bronze table
df_cust = spark.table(f"{CATALOG}.{SCHEMA}.bronze_customer_master")

# For now, simple copy (you can add SCD Type 2 logic later)
df_cust.write.format("delta").mode("overwrite") \
    .saveAsTable(f"{CATALOG}.{SCHEMA}.silver_customer_360")

print(f"✅ Created silver_customer_360")
print(f"📊 Total customers: {df_cust.count()}")

# Verify both tables
print("\n📊 Sample validated transactions:")
spark.sql(f"SELECT * FROM {CATALOG}.{SCHEMA}.silver_validated_transactions LIMIT 5").show()

print("\n📊 Sample customer 360:")
spark.sql(f"SELECT * FROM {CATALOG}.{SCHEMA}.silver_customer_360 LIMIT 5").show()

✅ Created silver_validated_transactions
📊 Valid transactions: 1000
✅ Created silver_customer_360
📊 Total customers: 100

📊 Sample validated transactions:
+--------------+-----------+----------------+----------------+--------+--------+
|transaction_id|customer_id|transaction_date|transaction_type|  amount|currency|
+--------------+-----------+----------------+----------------+--------+--------+
|         T0001|       C082|      2024-01-11|         Deposit|18111.14|     GBP|
|         T0002|       C080|      2024-01-25|         Deposit|16885.22|     EUR|
|         T0003|       C100|      2024-01-23|         Deposit|10247.88|     USD|
|         T0004|       C032|      2024-01-16|         Deposit|14207.33|     GBP|
|         T0005|       C054|      2024-01-06|         Deposit| 8556.78|     GBP|
+--------------+-----------+----------------+----------------+--------+--------+


📊 Sample customer 360:
+-----------+----------+---------+-------------+---------------+------------+
|customer_id|f